# 📊 Семинар: Линейная регрессия

Сегодня мы научимся:
- Загружать и обрабатывать данные для машинного обучения
- Понимать, зачем нужна нормализация и стандартизация
- Кодировать категориальные признаки (One-Hot Encoding)
- Разделять данные на train/val/test
- Обучать модель линейной регрессии
- Оценивать качество модели с помощью метрик


## 1. Загрузка данных

Мы будем работать с датасетом о зарплатах сотрудников. В нём содержится информация о возрасте, поле, уровне образования, должности, опыте работы и зарплате.

Наша задача — **предсказать зарплату** (Salary) на основе остальных признаков.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# URL датасета
url = "https://raw.githubusercontent.com/lyubovchubarova/MLHSELyceum2026/refs/heads/main/03_linreg/Salary_Data.csv"


### Задание 1.1
Загрузите данные по ссылке выше в DataFrame с помощью `pd.read_csv()` и выведите первые 10 строк.


In [ ]:
### ваш код здесь



### Задание 1.2
Посмотрите на размер датасета (количество строк и столбцов) и типы данных в каждом столбце.


In [ ]:
### ваш код здесь



---
## 2. Исследование и обработка данных (EDA)

Прежде чем обучать модель, нужно понять, с какими данными мы работаем:
- Есть ли пропущенные значения?
- Есть ли выбросы?
- Какие признаки числовые, а какие категориальные?


### Задание 2.1: Пропущенные значения
Проверьте, есть ли в датасете пропущенные значения. Используйте методы `.isnull()` и `.sum()`.


In [ ]:
### ваш код здесь



### Задание 2.2: Базовая статистика
Выведите описательную статистику для числовых столбцов с помощью `.describe()`. 

Обратите внимание на:
- Минимальные и максимальные значения (нет ли странных значений?)
- Среднее и медиану (сильно ли они отличаются?)


In [ ]:
### ваш код здесь



### Задание 2.3: Визуализация распределений
Постройте гистограммы для числовых признаков (`Age`, `Years of Experience`, `Salary`), чтобы увидеть их распределение.

Подсказка: используйте `plt.hist()` или `df['column'].hist()`


In [ ]:
### ваш код здесь



### Задание 2.4: Поиск выбросов
Постройте boxplot (ящик с усами) для столбца `Salary`, чтобы визуально определить выбросы.

Подсказка: `plt.boxplot(df['Salary'])`


In [ ]:
### ваш код здесь



### Задание 2.5: Категориальные признаки
Посмотрите на уникальные значения в категориальных столбцах (`Gender`, `Education Level`, `Job Title`).

Используйте `.unique()` или `.value_counts()`


In [ ]:
### ваш код здесь



---
## 3. Нормализация и стандартизация

### Зачем это нужно?

В линейной регрессии признаки могут иметь разные масштабы. Например:
- Возраст: 20-60 лет
- Зарплата: 30,000-250,000

Если мы не приведём признаки к одному масштабу, то:
1. **Коэффициенты модели будут трудно интерпретировать** — признак с большими значениями будет иметь маленький коэффициент, и наоборот
2. **Градиентный спуск может работать медленнее** — оптимизатор будет "скакать" по разным направлениям
3. **Регуляризация будет работать некорректно** — она будет сильнее штрафовать признаки с маленькими коэффициентами

### Два основных метода:

#### 1. Min-Max нормализация (MinMaxScaler)
Приводит значения к диапазону [0, 1]:

$$X_{norm} = \frac{X - X_{min}}{X_{max} - X_{min}}$$

#### 2. Стандартизация (StandardScaler)
Приводит к среднему = 0 и стандартному отклонению = 1:

$$X_{std} = \frac{X - \mu}{\sigma}$$

где $\mu$ — среднее, $\sigma$ — стандартное отклонение.

### Когда что использовать?
- **StandardScaler** — когда данные примерно нормально распределены (чаще используется)
- **MinMaxScaler** — когда нужен конкретный диапазон значений или данные не имеют выбросов


### Пример из документации sklearn

```python
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Стандартизация
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Нормализация
minmax = MinMaxScaler()
X_normalized = minmax.fit_transform(X)
```

**Важно!** 
- `fit_transform()` используем на тренировочных данных — он запоминает параметры (среднее, std, min, max)
- `transform()` используем на тестовых данных — применяет уже запомненные параметры


### Задание 3.1
Примените `StandardScaler` к числовым признакам (`Age`, `Years of Experience`). 

Выведите первые 5 значений до и после стандартизации, чтобы увидеть разницу.


In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

### ваш код здесь



---
## 4. One-Hot Encoding (кодирование категориальных признаков)

Модели машинного обучения работают с числами, а у нас есть текстовые признаки (`Gender`, `Education Level`). 

**One-Hot Encoding** — это способ преобразовать категориальный признак в набор бинарных (0/1) столбцов.

### Пример:
Было:
| Gender |
|--------|
| Male   |
| Female |
| Male   |

Стало:
| Gender_Male | Gender_Female |
|-------------|---------------|
| 1           | 0             |
| 0           | 1             |
| 1           | 0             |

### Пример из документации:

```python
# Способ 1: через pandas
df_encoded = pd.get_dummies(df, columns=['Gender', 'Education Level'])

# Способ 2: через sklearn
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
encoded = encoder.fit_transform(df[['Gender', 'Education Level']])
```


### Задание 4.1
Примените One-Hot Encoding к столбцам `Gender` и `Education Level` с помощью `pd.get_dummies()`.

**Примечание:** столбец `Job Title` содержит слишком много уникальных значений, поэтому его лучше пока убрать из датасета (или использовать другие методы кодирования, но это уже advanced тема).


In [ ]:
### ваш код здесь



---
## 5. Разделение на признаки (X) и целевую переменную (y)

В машинном обучении принято разделять данные на:
- **X** (features, признаки) — то, на основе чего мы делаем предсказание
- **y** (target, целевая переменная) — то, что мы предсказываем

### Почему это важно?
1. **Модель не должна "видеть" ответ** — если мы случайно оставим целевую переменную в признаках, модель просто выучит "подглядывать"
2. **Удобство работы** — большинство функций sklearn принимают X и y отдельно
3. **Стандарт в индустрии** — так делают все, и вам будет проще читать чужой код

### Соглашения по именованию:
- `X` — признаки (большая буква, потому что это матрица)
- `y` — целевая переменная (маленькая буква, потому что это вектор)


### Задание 5.1
Разделите ваш закодированный датасет на X и y:
- **y** — столбец `Salary`
- **X** — все остальные столбцы (кроме `Salary` и `Job Title`)


In [ ]:
### ваш код здесь



---
## 6. Разделение на train / validation / test

### Почему нельзя просто обучить на всех данных?

Если мы обучим модель на всех данных и проверим её на тех же данных, мы не узнаем, как она будет работать на **новых** данных. Модель может просто "запомнить" обучающую выборку (переобучение).

### Зачем три части, а не две?

- **Train (тренировочная выборка)** — на ней обучаем модель
- **Validation (валидационная выборка)** — на ней подбираем гиперпараметры и сравниваем разные модели
- **Test (тестовая выборка)** — финальная проверка, смотрим только один раз в самом конце

**Почему validation отдельно от test?**

Когда мы подбираем гиперпараметры на валидации, мы косвенно "подглядываем" в эти данные. Если использовать для этого тест, то оценка на тесте будет завышена. Поэтому тест храним "в тайне" до самого конца.

### Типичные пропорции:
- Train: 60-80%
- Validation: 10-20%
- Test: 10-20%

### Что такое random_state (seed)?

При разделении данные перемешиваются случайным образом. `random_state` — это "зерно" генератора случайных чисел. 

**Зачем это нужно?**
- Чтобы результаты были **воспроизводимыми** — при одном и том же seed получим одно и то же разбиение
- Чтобы можно было **сравнивать** эксперименты — если seed разный, то и данные разные, сравнивать некорректно


### Пример из документации sklearn:

```python
from sklearn.model_selection import train_test_split

# Сначала отделяем тест
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, 
    test_size=0.2,      # 20% на тест
    random_state=42     # для воспроизводимости
)

# Потом из оставшегося отделяем валидацию
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val,
    test_size=0.25,     # 25% от 80% = 20% от всего
    random_state=42
)

# Итого: train=60%, val=20%, test=20%
```

### Соглашения по именованию:
- `X_train`, `y_train` — тренировочные данные
- `X_val`, `y_val` — валидационные данные
- `X_test`, `y_test` — тестовые данные


### Задание 6.1
Разделите данные на train (60%), validation (20%) и test (20%).

Используйте `random_state=42` для воспроизводимости.

Выведите размеры каждой выборки, чтобы убедиться, что всё правильно.


In [ ]:
from sklearn.model_selection import train_test_split

### ваш код здесь



---
## 7. Обучение модели линейной регрессии

Теперь мы готовы обучить модель! В sklearn это делается очень просто.

### Пример из документации:

```python
from sklearn.linear_model import LinearRegression

# Создаём модель
model = LinearRegression()

# Обучаем на тренировочных данных
model.fit(X_train, y_train)

# Делаем предсказания
y_pred = model.predict(X_test)
```


### Задание 7.1
Создайте модель линейной регрессии и обучите её на тренировочных данных.


In [ ]:
from sklearn.linear_model import LinearRegression

### ваш код здесь



### Задание 7.2
Сделайте предсказания на валидационной выборке (`X_val`).


In [ ]:
### ваш код здесь



---
## 8. Метрики качества регрессии

Как понять, хорошо ли работает наша модель? Для этого существуют метрики.

### MAE (Mean Absolute Error) — средняя абсолютная ошибка

$$MAE = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|$$

**Интерпретация:** в среднем мы ошибаемся на MAE единиц (в нашем случае — долларов).

**Плюсы:** 
- Легко интерпретировать
- Устойчива к выбросам (по сравнению с MSE)

---

### MSE (Mean Squared Error) — средняя квадратичная ошибка

$$MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$$

**Интерпретация:** среднее значение квадратов ошибок. Большие ошибки "штрафуются" сильнее.

**Плюсы:**
- Дифференцируема (важно для оптимизации)
- Сильнее штрафует большие ошибки

**Минусы:**
- Размерность — квадрат исходной (доллары²), сложно интерпретировать

---

### RMSE (Root Mean Squared Error) — корень из MSE

$$RMSE = \sqrt{MSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}$$

**Интерпретация:** как MAE, но сильнее штрафует большие ошибки. Имеет ту же размерность, что и y (доллары).

---

### R² (коэффициент детерминации)

$$R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}$$

**Интерпретация:** доля дисперсии целевой переменной, которую объясняет модель.

- R² = 1 — модель идеально предсказывает
- R² = 0 — модель предсказывает не лучше, чем просто среднее значение
- R² < 0 — модель предсказывает хуже, чем среднее (что-то пошло не так)

**Плюсы:**
- Безразмерная метрика (от 0 до 1 в норме)
- Легко сравнивать разные модели


### Пример из документации sklearn:

```python
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = mean_squared_error(y_true, y_pred, squared=False)  # или np.sqrt(mse)
r2 = r2_score(y_true, y_pred)

print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.4f}")
```


### Задание 8.1
Посчитайте все 4 метрики (MAE, MSE, RMSE, R²) для предсказаний на **валидационной** выборке.


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

### ваш код здесь



### Задание 8.2
Теперь посчитайте те же метрики на **тестовой** выборке.

Сравните результаты с валидацией — сильно ли они отличаются?


In [ ]:
### ваш код здесь



---
## 9. Визуализация результатов

Графики помогают лучше понять, как работает модель.


### Задание 9.1: Предсказания vs Реальные значения

Постройте scatter plot, где:
- По оси X — реальные значения зарплаты (y_test)
- По оси Y — предсказанные значения (y_pred на тесте)

Если модель идеальна, все точки будут лежать на диагонали y=x.

Подсказка: добавьте диагональную линию для сравнения с помощью `plt.plot([min, max], [min, max], 'r--')`


In [ ]:
### ваш код здесь



### Задание 9.2: Распределение ошибок (residuals)

Постройте гистограмму ошибок (разница между реальным и предсказанным значением).

В идеале ошибки должны быть распределены нормально вокруг нуля.


In [ ]:
### ваш код здесь



---
## 10. Выводы

### Задание 10.1
Напишите краткие выводы по результатам работы:

1. Какой R² получился? Это хороший результат?
2. Какая средняя ошибка в предсказании зарплаты (MAE)?
3. Что можно было бы улучшить в модели?
4. Какие признаки, по вашему мнению, больше всего влияют на зарплату?


*Напишите ваши выводы здесь:*

1. ...
2. ...
3. ...
4. ...


---
## 🎯 Бонусное задание (если осталось время)

Посмотрите на коэффициенты модели и определите, какие признаки оказывают наибольшее влияние на предсказание зарплаты.

Подсказка: `model.coef_` — коэффициенты, `model.feature_names_in_` или названия столбцов X


In [ ]:
### ваш код здесь

